In [1]:
!git clone https://github.com/AleksL04/ML-Project.git
%cd ML-Project

Cloning into 'ML-Project'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 24 (delta 8), reused 21 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (24/24), 15.57 KiB | 15.57 MiB/s, done.
Resolving deltas: 100% (8/8), done.
/content/ML-Project


In [14]:
!git pull

Already up to date.


In [2]:
%%capture
!pip install --upgrade gensim
!gdown --folder 1rl_TJMHtcP-S0fTHNUpr9oi8Ndz2Y5z4 -O project_data
!gdown --folder 1RmC_We2lfsJontwrxlc4dzqEY_mNy-UW -O word2vec_model

In [8]:
ls -l

total 104
-rw-r--r-- 1 root root 81531 Dec 10 21:27 Ensemble_Models.ipynb
-rw-r--r-- 1 root root  2836 Dec 10 21:27 main_console.ipynb
drwxr-xr-x 2 root root  4096 Dec 10 21:27 model_code/
-rw-r--r-- 1 root root  1552 Dec 10 21:27 process_df.py
drwxr-xr-x 2 root root  4096 Dec 10 21:28 project_data/
drwxr-xr-x 2 root root  4096 Dec 10 21:27 __pycache__/
drwxr-xr-x 2 root root  4096 Dec 10 21:28 word2vec_model/


In [ ]:
import pandas as pd
import numpy as np

In [4]:
train_df = pd.read_csv('project_data/train.csv')
valid_df = pd.read_csv('project_data/valid.csv')
test_df = pd.read_csv('project_data/test.csv')

In [5]:
from process_df import process_df

X_train = process_df(train_df)
y_train = train_df['label'].values

X_valid = process_df(valid_df)
y_valid = valid_df['label'].values

X_test = process_df(test_df)
y_test = test_df['label'].values

In [ ]:
from model_code.LSTM_code import trainLSTM

LSTM_model = trainLSTM(X_train, y_train, X_valid, y_valid)

test_loss, test_acc = LSTM_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

In [ ]:
from model_code.CNN_code import trainCNN

CNN_model = trainCNN(X_train, y_train, X_valid, y_valid)

test_loss, test_acc = CNN_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8886 - loss: 0.2669
Test Accuracy: 0.8830


In [ ]:
val_pred_LSTM = LSTM_model.predict(X_valid)
val_pred_CNN = CNN_model.predict(X_valid)

X_meta_train = np.column_stack((val_pred_LSTM, val_pred_CNN))

y_meta_train = y_valid

In [ ]:
from model_code.Blender_code import trainBlender

Blender_model = trainBlender(X_meta_train, y_meta_train)

In [ ]:
test_pred_LSTM = LSTM_model.predict(X_test)
test_pred_CNN = CNN_model.predict(X_test)

X_meta_test = np.column_stack((test_pred_LSTM, test_pred_CNN))

final_predictions = Blender_model.predict(X_meta_test)

test_loss, test_acc = Blender_model.evaluate(X_meta_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")